## Filtering example using dates

Let us understand how to filter the data using dates leveraging appropriate date manipulation functions.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.spark.com/plans) to learn Spark SQL using our unique integrated LMS.

In [1]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Basic Transformations'). \
    master('yarn'). \
    getOrCreate()

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

### Tasks

Let us perform some tasks to understand filtering in detail. Solve all the problems by passing  conditions using both SQL Style as well as API Style.

* Read the data for the month of 2008 January. We will be using only 2008 January data for the demos.

In [2]:
airtraffic_path = "/public/airtraffic_all/airtraffic-part/flightmonth=200801"

In [3]:
airtraffic = spark. \
    read. \
    parquet(airtraffic_path)

In [4]:
airtraffic.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Car

In [5]:
airtraffic.select('Year', 'Month', 'DayOfMonth').distinct().show(31)

+----+-----+----------+
|Year|Month|DayOfMonth|
+----+-----+----------+
|2008|    1|        28|
|2008|    1|        25|
|2008|    1|        20|
|2008|    1|        11|
|2008|    1|         4|
|2008|    1|         5|
|2008|    1|        15|
|2008|    1|         3|
|2008|    1|        16|
|2008|    1|         9|
|2008|    1|        17|
|2008|    1|        19|
|2008|    1|        12|
|2008|    1|         6|
|2008|    1|        21|
|2008|    1|        18|
|2008|    1|         7|
|2008|    1|         1|
|2008|    1|        26|
|2008|    1|        24|
|2008|    1|        23|
|2008|    1|         8|
|2008|    1|         2|
|2008|    1|        14|
|2008|    1|        10|
|2008|    1|        13|
|2008|    1|        27|
|2008|    1|        29|
|2008|    1|        30|
|2008|    1|        22|
|2008|    1|        31|
+----+-----+----------+



In [6]:
airtraffic.select('Year', 'Month', 'DayOfMonth').distinct().count()

31

In [7]:
airtraffic.count()

605659

* Get number of flights departed late on Sundays.

In [8]:
l = [('X',)]

In [9]:
df = spark.createDataFrame(l, "dummy STRING")

In [10]:
df.show()

+-----+
|dummy|
+-----+
|    X|
+-----+



In [11]:
from pyspark.sql.functions import current_date

In [12]:
df.select(current_date()).show()

+--------------+
|current_date()|
+--------------+
|    2021-03-01|
+--------------+



In [16]:
from pyspark.sql.functions import date_format

In [17]:
df.select(current_date().alias('current_date'), date_format(current_date(), 'EE').alias('day_name')).show()

+------------+--------+
|current_date|day_name|
+------------+--------+
|  2021-03-01|     Mon|
+------------+--------+



In [18]:
df.select(current_date().alias('current_date'), date_format(current_date(), 'EEEE').alias('day_name')).show()

+------------+--------+
|current_date|day_name|
+------------+--------+
|  2021-03-01|  Monday|
+------------+--------+



In [19]:
airtraffic.select('Year', 'Month', 'DayOfMonth').show()

+----+-----+----------+
|Year|Month|DayOfMonth|
+----+-----+----------+
|2008|    1|        16|
|2008|    1|        17|
|2008|    1|        17|
|2008|    1|        17|
|2008|    1|        17|
|2008|    1|        18|
|2008|    1|        18|
|2008|    1|        19|
|2008|    1|        20|
|2008|    1|        20|
|2008|    1|        21|
|2008|    1|        21|
|2008|    1|        21|
|2008|    1|        21|
|2008|    1|        21|
|2008|    1|        22|
|2008|    1|        22|
|2008|    1|        23|
|2008|    1|        23|
|2008|    1|        23|
+----+-----+----------+
only showing top 20 rows



In [20]:
from pyspark.sql.functions import col, lpad, concat

In [21]:
airtraffic.select(
    concat(
        col("Year"),
        lpad(col("Month"), 2, "0"),
        lpad(col("DayOfMonth"), 2, "0")
    ).alias('FlightDate')
).show()

+----------+
|FlightDate|
+----------+
|  20080116|
|  20080117|
|  20080117|
|  20080117|
|  20080117|
|  20080118|
|  20080118|
|  20080119|
|  20080120|
|  20080120|
|  20080121|
|  20080121|
|  20080121|
|  20080121|
|  20080121|
|  20080122|
|  20080122|
|  20080123|
|  20080123|
|  20080123|
+----------+
only showing top 20 rows



In [22]:
airtraffic. \
    select(
        concat(
            col("Year"),
            lpad(col("Month"), 2, "0"),
            lpad(col("DayOfMonth"), 2, "0")
        ).alias('FlightDate')
    ). \
    selectExpr("to_date(FlightDate, 'yyyyMMdd') AS FlightDate"). \
    show()

+----------+
|FlightDate|
+----------+
|2008-01-16|
|2008-01-17|
|2008-01-17|
|2008-01-17|
|2008-01-17|
|2008-01-18|
|2008-01-18|
|2008-01-19|
|2008-01-20|
|2008-01-20|
|2008-01-21|
|2008-01-21|
|2008-01-21|
|2008-01-21|
|2008-01-21|
|2008-01-22|
|2008-01-22|
|2008-01-23|
|2008-01-23|
|2008-01-23|
+----------+
only showing top 20 rows



In [23]:
airtraffic. \
    select(
        concat(
            col("Year"),
            lpad(col("Month"), 2, "0"),
            lpad(col("DayOfMonth"), 2, "0")
        ).alias('FlightDate')
    ). \
    selectExpr("date_format(to_date(FlightDate, 'yyyyMMdd'), 'EEEE') AS FlightDay"). \
    show()

+---------+
|FlightDay|
+---------+
|Wednesday|
| Thursday|
| Thursday|
| Thursday|
| Thursday|
|   Friday|
|   Friday|
| Saturday|
|   Sunday|
|   Sunday|
|   Monday|
|   Monday|
|   Monday|
|   Monday|
|   Monday|
|  Tuesday|
|  Tuesday|
|Wednesday|
|Wednesday|
|Wednesday|
+---------+
only showing top 20 rows



In [24]:
from pyspark.sql.functions import date_format, to_date

In [25]:
airtraffic. \
    select(
        concat(
            col("Year"),
            lpad(col("Month"), 2, "0"),
            lpad(col("DayOfMonth"), 2, "0")
        ).alias('FlightDate')
    ). \
    select(date_format(to_date('FlightDate', 'yyyyMMdd'), 'EEEE').alias('FlightDay')). \
    show()

+---------+
|FlightDay|
+---------+
|Wednesday|
| Thursday|
| Thursday|
| Thursday|
| Thursday|
|   Friday|
|   Friday|
| Saturday|
|   Sunday|
|   Sunday|
|   Monday|
|   Monday|
|   Monday|
|   Monday|
|   Monday|
|  Tuesday|
|  Tuesday|
|Wednesday|
|Wednesday|
|Wednesday|
+---------+
only showing top 20 rows



In [26]:
from pyspark.sql.functions import col, concat, lpad

In [28]:
airtraffic. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter("""
           date_format(to_date(FlightDate, 'yyyyMMdd'), 'EEEE') = 'Sunday'
           """). \
    count()

76395

In [27]:
airtraffic. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter("""
           IsDepDelayed = 'YES' AND 
           date_format(to_date(FlightDate, 'yyyyMMdd'), 'EEEE') = 'Sunday'
           """). \
    count()

34708

* API Style

In [29]:
from pyspark.sql.functions import col, concat, lpad, \
    date_format, to_date

In [30]:
airtraffic. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter((col("IsDepDelayed") == "YES") &
           (date_format(
               to_date("FlightDate", "yyyyMMdd"), "EEEE"
           ) == "Sunday")
          ). \
    count()

34708